<a href="https://colab.research.google.com/github/sh-0620/dacon-recommender-system/blob/main/recommendations_TfidfVectorizer(all)_content_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

# CSV 파일 로드
log_df = pd.read_csv("/content/drive/MyDrive/웹 기사 추천시스템/view_log.csv")  # 유저 로그 데이터
article__df = pd.read_csv("/content/drive/MyDrive/웹 기사 추천시스템/article_info.csv")  # 기사 데이터
article_nouns_df = pd.read_csv("/content/drive/MyDrive/웹 기사 추천시스템/article_nouns.csv")  # 기사 데이터

In [ ]:
article_nouns_df.head()

In [ ]:
article_nouns_df = article_nouns_df.drop(['userCountry', 'userRegion', 'userID'], axis=1)

In [ ]:
merged_df = pd.merge(log_df, article_nouns_df, on='articleID')

In [ ]:
# 기사 데이터 중 로그 데이터에 없는 기사 제거
article_nouns_df  = article_nouns_df[article_nouns_df['articleID'].isin(merged_df['articleID'].unique())]

In [ ]:
# 라벨 인코더 생성 및 학습
article_label_encoder = LabelEncoder()
merged_df['articleID'] = article_label_encoder.fit_transform(merged_df['articleID'])
article_nouns_df['articleID'] = article_label_encoder.transform(article_nouns_df['articleID'])

In [ ]:
# 유저 아이디 라벨 인코딩 적용
user_label_encoder = LabelEncoder()
merged_df['userID'] = user_label_encoder.fit_transform(merged_df['userID'])

In [ ]:
# log_df 원핫 인코딩
encoder = OneHotEncoder()

# 범주형 변수 선택 (지역, 나라)
log_df_categories = log_df[['userRegion', 'userCountry']]

# One-Hot Encoding 수행
log_df_encoded_categories = encoder.fit_transform(log_df_categories).toarray()

In [ ]:
# article_nouns_df 원핫 인코딩
encoder = OneHotEncoder()

# 범주형 변수 선택 (언어, 형식)
article_nouns_df_categories = article_nouns_df[['Format', 'Language']]

# One-Hot Encoding 수행
article_nouns_df_encoded_categories = encoder.fit_transform(article_nouns_df_categories).toarray()

In [ ]:
# 기사 데이터 전처리: 제목 + 본문
tfidf = TfidfVectorizer(
    stop_words='english',
    max_features=10000,
    ngram_range=(1, 2),
    max_df=0.8,
    min_df=3,
    sublinear_tf=True,
    smooth_idf=True
)
tfidf_matrix = tfidf.fit_transform(article_nouns_df['Title'] + ' ' + article_nouns_df['Content'])

In [ ]:
# TF-IDF 행렬, article_nouns_df 원핫 인코딩 / log_df 원핫 인코딩 / => 피처 벡터 결합
combined_features = np.hstack((tfidf_matrix.toarray(), article_nouns_df_encoded_categories))


In [ ]:
# 유저별 기사 읽기 패턴 추출
user_read_articles = defaultdict(list)
for _, row in merged_df.iterrows():
    user_read_articles[row['userID']].append(row['articleID'])

In [ ]:
# 유저 기반 협업 필터링 모델
def recommend_articles(userID, top_n=5):
    if userID not in user_read_articles or not user_read_articles[userID]:

        return article_nouns_df.sample(n=top_n)[['Title', 'Content']]


    read_articles = user_read_articles[userID]
    read_articles_features = combined_features[read_articles]

    if read_articles_features.shape[0] == 0:
        return article_nouns_df.sample(n=top_n)[['Title', 'Content']]

    # 사용자가 읽은 기사들의 피처 벡터 평균 계산
    avg_features = np.mean(read_articles_features, axis=0).reshape(1, -1)

    # 사용자가 읽은 기사들의 피처 벡터 평균과 모든 기사들의 피처 벡터 간의 코사인 유사도 계산 (콘텐츠 기반 필터링)
    content_similarities = cosine_similarity(avg_features, combined_features).flatten()

    # 모든 사용자의 읽은 기사 정보를 고려하여 협업 필터링 기반 유사도 계산
    collaborative_similarities = np.zeros(len(combined_features))
    for other_user, other_user_read_articles in user_read_articles.items():
        if other_user == userID:
            continue
        # 다른 사용자가 읽은 기사들과 현재 사용자의 읽은 기사의 피처 벡터 간의 유사도 계산
        other_user_features = combined_features[other_user_read_articles]
        other_user_avg_features = np.mean(other_user_features, axis=0).reshape(1, -1)
        collaborative_similarities += cosine_similarity(other_user_avg_features, combined_features).flatten()

    # 콘텐츠 기반 필터링과 협업 필터링의 유사도 합산 (가중치 적용 가능)
    hybrid_similarities = content_similarities + collaborative_similarities

    # # 사용자가 읽은 기사들의 피처 벡터 평균과 모든 기사들의 피처 벡터 간의 코사인 유사도 계산
    # similarities = cosine_similarity(avg_features, combined_features)

    # 유사도가 높은 상위 top_n 개의 기사 인덱스 추출
    similar_articles_indices = hybrid_similarities.argsort()[-top_n:][::-1]

    # 추천 기사 반환 (유저가 읽은 기사 포함)
    recommendations = article_nouns_df.iloc[similar_articles_indices]
    return recommendations[['Title', 'Content']]

In [ ]:
user_recommendations = []

for encoded_user_id in merged_df['userID'].unique():
    # 추천 기사 인덱스 가져오기
    recommended_articles = recommend_articles(encoded_user_id)

    # 원래 유저 아이디로 역변환
    original_user_id = user_label_encoder.inverse_transform([encoded_user_id])[0]

    for idx in recommended_articles.index:
        # 원래 기사 아이디로 역변환
        original_article_id = article_label_encoder.inverse_transform([article_nouns_df.loc[idx, 'articleID']])[0]

        # 추천 결과를 리스트에 저장
        user_recommendations.append({
            'userID': original_user_id,
            'articleID': original_article_id
        })

In [ ]:
print(user_recommendations)

In [ ]:
recommendations_df = pd.DataFrame(user_recommendations)
recommendations_df.to_csv('user_recommendations_(all)_content_tune_ collabo.csv', index=False)